In [35]:
D = [20, 30, 50, 50] # Demand
S = [40, 60, 50] # Supply capacities

n = len(D) # number of demand points
m = len(S) # number of supply points

# cost matrix - cost of shipping from demander i to supplier j
C = [[4, 6, 8, 8],
     [6, 8, 6, 7],
     [5, 7, 6, 8]]

In [17]:
def print_results(X, total_cost):
    print("Shipping matrix:")
    for row in X:
        print(row)
    print("Total cost:", total_cost)

In [18]:
def north_west_corner(cost_matrix, S, D, print_steps=False):
    """ 
    North-West Corner Method: 
        - Allocate shipping amounts starting from the top-left corner of the cost matrix when possible
        - Delete row/column when the capacity of a supplier/DC is reached
        - Repeat until all suppliers/DCs are satisfied

    Args:
        - cost_matrix: cost matrix of shipping - size m x n (m: number of suppliers, n: number of DCs)
        - print_steps: if True, print the steps of the algorithm
    
    Returns:
        - X: shipping matrix
        - total_cost: total cost of shipping
    """
    S_cap = S.copy()
    D_cap = D.copy()

    i = 0
    j = 0
    X = [[0 for _ in range(n)] for _ in range(m)]
    total_cost = 0

    while i < m and j < n:
        if S_cap[i] > 0 and D_cap[j] > 0:
            X[i][j] = min(S_cap[i], D_cap[j])
            S_cap[i] -= X[i][j]
            D_cap[j] -= X[i][j]
            total_cost += X[i][j] * cost_matrix[i][j]
            if print_steps:
                print("Shipping amount from DC", j, "to supplier", i, ":", X[i][j])
                print("Remaining capacity of DC", j, ":", D_cap[j])
                print("Remaining capacity of supplier", i, ":", S_cap[i])
                print("Total cost:", total_cost, "\n")

            if S_cap[i] == 0:
                i += 1
            else:
                j += 1
        else:
            if S_cap[i] == 0:
                i += 1
            else:
                j += 1

    return X, total_cost

X, total_cost = north_west_corner(C, S, D, print_steps=True)
print_results(X, total_cost)


Shipping amount from DC 0 to supplier 0 : 20
Remaining capacity of DC 0 : 0
Remaining capacity of supplier 0 : 20
Total cost: 80 

Shipping amount from DC 1 to supplier 0 : 20
Remaining capacity of DC 1 : 10
Remaining capacity of supplier 0 : 0
Total cost: 200 

Shipping amount from DC 1 to supplier 1 : 10
Remaining capacity of DC 1 : 0
Remaining capacity of supplier 1 : 50
Total cost: 280 

Shipping amount from DC 2 to supplier 1 : 50
Remaining capacity of DC 2 : 0
Remaining capacity of supplier 1 : 0
Total cost: 580 

Shipping amount from DC 3 to supplier 2 : 50
Remaining capacity of DC 3 : 0
Remaining capacity of supplier 2 : 0
Total cost: 980 

Shipping matrix:
[20, 20, 0, 0]
[0, 10, 50, 0]
[0, 0, 0, 50]
Total cost: 980


In [19]:
def min_cost_method(cost_matrix, S, D, print_steps=False):
    """ 
    Minimum Cost Method: 
        - Find the minimum cost cell in the cost matrix
        - Allocate the maximum possible amount of shipping from the corresponding DC to the supplier
        - Delete row/column when the capacity of a supplier/DC is reached
        - Repeat until all suppliers/DCs are satisfied

    Args:
        - cost_matrix: cost matrix of shipping - size m x n (m: number of suppliers, n: number of DCs)
        - print_steps: if True, print the steps of the algorithm
    
    Returns:
        - X: shipping matrix
        - total_cost: total cost of shipping
    """
    S_cap = S.copy()
    D_cap = D.copy()

    X = [[0 for _ in range(n)] for _ in range(m)]
    total_cost = 0

    while True:
        min_cost = float('inf')
        min_i = -1
        min_j = -1

        for i in range(m):
            for j in range(n):
                if S_cap[i] > 0 and D_cap[j] > 0 and cost_matrix[i][j] < min_cost:
                    min_cost = cost_matrix[i][j]
                    min_i = i
                    min_j = j

        if min_i == -1 or min_j == -1:
            break

        X[min_i][min_j] = min(S_cap[min_i], D_cap[min_j])
        S_cap[min_i] -= X[min_i][min_j]
        D_cap[min_j] -= X[min_i][min_j]
        total_cost += X[min_i][min_j] * cost_matrix[min_i][min_j]

        if print_steps:
            print("Shipping amount from DC", min_j, "to supplier", min_i, ":", X[min_i][min_j])
            print("Total cost:", total_cost, "\n")

    return X, total_cost


X, total_cost = min_cost_method(C, S, D, print_steps=True)
print_results(X, total_cost)

Shipping amount from DC 0 to supplier 0 : 20
Total cost: 80 

Shipping amount from DC 1 to supplier 0 : 20
Total cost: 200 

Shipping amount from DC 2 to supplier 1 : 50
Total cost: 500 

Shipping amount from DC 3 to supplier 1 : 10
Total cost: 570 

Shipping amount from DC 1 to supplier 2 : 10
Total cost: 640 

Shipping amount from DC 3 to supplier 2 : 40
Total cost: 960 

Shipping matrix:
[20, 20, 0, 0]
[0, 0, 50, 10]
[0, 10, 0, 40]
Total cost: 960


In [20]:
D = [20, 30, 50, 50] # DCs capacities
S = [40, 60, 50] # Suppliers capacities

n = len(D) # number of DCs
m = len(S) # number of suppliers

# cost matrix - cost of shipping from DC i to supplier j
C = [[4, 6, 8, 8],
     [6, 8, 6, 7],
     [5, 7, 6, 8]]

In [22]:
import numpy as np

def vogel_approximation_method(cost_matrix, supply, demand, print_steps=False):
    supply = supply.copy()
    demand = demand.copy()
    n_rows, n_cols = cost_matrix.shape
    allocation = np.zeros((n_rows, n_cols))
    transportation_cost = 0
    
    while np.any(supply) and np.any(demand):
        row_penalty = np.zeros(n_rows)
        col_penalty = np.zeros(n_cols)
        
        for i in range(n_rows):
            sorted_row = np.sort(cost_matrix[i, :])
            row_penalty[i] = sorted_row[1] - sorted_row[0] if len(sorted_row) > 1 else sorted_row[0]
        
        for j in range(n_cols):
            sorted_col = np.sort(cost_matrix[:, j])
            col_penalty[j] = sorted_col[1] - sorted_col[0] if len(sorted_col) > 1 else sorted_col[0]
        
        row_max_penalty = np.max(row_penalty)
        col_max_penalty = np.max(col_penalty)
        
        if row_max_penalty > col_max_penalty:
            i = np.argmax(row_penalty)
            j = np.argmin(cost_matrix[i, :])
        else:
            j = np.argmax(col_penalty)
            i = np.argmin(cost_matrix[:, j])
        
        allocation_amount = min(supply[i], demand[j])
        allocation[i, j] = allocation_amount
        supply[i] -= allocation_amount
        demand[j] -= allocation_amount
        
        transportation_cost += allocation_amount * cost_matrix[i, j]
        cost_matrix[i, j] = 1e9  # Remove this cell for future consideration
        
        if print_steps:
            print(f"Allocated {allocation_amount} units to cell ({i}, {j})")
            print(f"Updated supply: {supply}")
            print(f"Updated demand: {demand}")
            print(f"Current transportation cost: {transportation_cost}\n")
    
    return allocation, transportation_cost

# Example usage:
cost_matrix = np.array(C)
supply = np.array(S)
demand = np.array(D)

allocation, transportation_cost = vogel_approximation_method(cost_matrix, supply, demand, print_steps=True)
print("Allocation:\n", allocation)
print("Total transportation cost:", transportation_cost)


Allocated 20 units to cell (0, 0)
Updated supply: [20 60 50]
Updated demand: [ 0 30 50 50]
Current transportation cost: 80

Allocated 20 units to cell (0, 1)
Updated supply: [ 0 60 50]
Updated demand: [ 0 10 50 50]
Current transportation cost: 200

Allocated 0 units to cell (2, 0)
Updated supply: [ 0 60 50]
Updated demand: [ 0 10 50 50]
Current transportation cost: 200

Allocated 0 units to cell (1, 0)
Updated supply: [ 0 60 50]
Updated demand: [ 0 10 50 50]
Current transportation cost: 200

Allocated 10 units to cell (2, 1)
Updated supply: [ 0 60 40]
Updated demand: [ 0  0 50 50]
Current transportation cost: 270

Allocated 0 units to cell (1, 1)
Updated supply: [ 0 60 40]
Updated demand: [ 0  0 50 50]
Current transportation cost: 270

Allocated 40 units to cell (2, 2)
Updated supply: [ 0 60  0]
Updated demand: [ 0  0 10 50]
Current transportation cost: 510

Allocated 0 units to cell (2, 3)
Updated supply: [ 0 60  0]
Updated demand: [ 0  0 10 50]
Current transportation cost: 510

Alloc

# 2. Check optimality
Using the Stepping stone method

In [29]:
import numpy as np

def find_loops(allocation, start):
    """
    Find all possible loops starting from the given cell.
    """
    def search_path(path, i, j):
        if len(path) > 1 and path[0] == path[-1]:
            loops.append(path[:-1])
            return
        for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            ni, nj = i + di, j + dj
            if 0 <= ni < allocation.shape[0] and 0 <= nj < allocation.shape[1] and allocation[ni, nj] > 0:
                if (ni, nj) not in path:
                    search_path(path + [(ni, nj)], ni, nj)

    loops = []
    search_path([start], start[0], start[1])
    return loops

def calculate_cost_change(loop, cost_matrix):
    """
    Calculate the net change in cost for a given loop.
    """
    change = 0
    sign = 1
    for i, j in loop:
        change += sign * cost_matrix[i, j]
        sign *= -1
    return change

def stepping_stone_method(allocation, cost_matrix, output_file="./output/stepping_stone_output.txt"):
    """
    Check the optimality of the current solution using the Stepping Stone Method,
    search for each possible improvement, and decide the optimal one.
    """
    num_rows, num_cols = allocation.shape
    u = [None] * num_rows
    v = [None] * num_cols
    potentials = np.zeros((num_rows, num_cols))
    u[0] = 0  # Start with the first row

    # Calculate the potentials using the initial basic feasible solution
    while None in u or None in v:
        for i in range(num_rows):
            for j in range(num_cols):
                if allocation[i, j] > 0:
                    if u[i] is not None and v[j] is None:
                        v[j] = cost_matrix[i, j] - u[i]
                    elif v[j] is not None and u[i] is None:
                        u[i] = cost_matrix[i, j] - v[j]

    # Calculate the reduced costs (potentials)
    for i in range(num_rows):
        for j in range(num_cols):
            if allocation[i, j] == 0:
                potentials[i, j] = cost_matrix[i, j] - (u[i] + v[j])

    # Search for improvements
    best_change = 0
    changes = []
    best_loop = None
    for i in range(num_rows):
        for j in range(num_cols):
            if allocation[i, j] == 0:
                loops = find_loops(allocation, (i, j))
                for loop in loops:
                    change = calculate_cost_change(loop, cost_matrix)
                    changes.append(((i, j), change))
                    print(f"Loop: {loop}, Change: {change}")
                    if change < best_change:
                        best_change = change
                        best_loop = loop

    # Write results to the output file
    with open(output_file, "w") as file:
        file.write("Stepping Stone Method Output\n")
        file.write("==================================\n")
        file.write("Initial Allocation:\n")
        for row in allocation:
            file.write(f"{row}\n")
        file.write("\n")
        file.write("Cost Matrix:\n")
        for row in cost_matrix:
            file.write(f"{row}\n")
        file.write("\n")
        file.write(f"Potentials:\n{potentials}\n")
        file.write(f"Best Cost Change: {best_change}\n")
        file.write(f"Best Loop: {best_loop}\n")

        if best_change < 0:
            file.write("The solution is not optimal. An improvement is possible.\n")
        else:
            file.write("The solution is optimal.\n")

In [36]:
D = [20, 30, 50, 50] # Demand
S = [40, 60, 50] # Supply capacities

n = len(D) # number of demand points
m = len(S) # number of supply points

# cost matrix - cost of shipping from demander i to supplier j
C = [[4, 6, 8, 8],  # Supplier 1 
     [6, 8, 6, 7],  # Supplier 2
     [5, 7, 6, 8]]  # Supplier 3
    #D1 D2 D3 D4

def get_results(C, S, D):
    min_cost_alloc, min_cost = min_cost_method(C, S, D, print_steps=False)
    north_west_alloc, north_west_cost = north_west_corner(C, S, D, print_steps=False)

    cost_matrix = np.array(C)
    supply = np.array(S)
    demand = np.array(D)
    vogel_alloc, transportation_cost = vogel_approximation_method(cost_matrix, supply, demand, print_steps=False)

    min_cost_alloc = np.array(min_cost_alloc)
    north_west_alloc = np.array(north_west_alloc)

    print("Minimum Cost Method:")
    print_results(min_cost_alloc, min_cost)
    print("\nNorth-West Corner Method:")
    print_results(north_west_alloc, north_west_cost)
    print("\nVogel's Approximation Method:")
    print_results(vogel_alloc, transportation_cost)

    return min_cost_alloc, north_west_alloc, vogel_alloc

min_cost_alloc, north_west_alloc, vogel_alloc = get_results(C, S, D)

Minimum Cost Method:
Shipping matrix:
[20 20  0  0]
[ 0  0 50 10]
[ 0 10  0 40]
Total cost: 960

North-West Corner Method:
Shipping matrix:
[20 20  0  0]
[ 0 10 50  0]
[ 0  0  0 50]
Total cost: 980

Vogel's Approximation Method:
Shipping matrix:
[20. 20.  0.  0.]
[ 0.  0. 10. 50.]
[ 0. 10. 40.  0.]
Total cost: 920


In [33]:
D = [20, 30, 50, 50] # Demand
S = [40, 60, 50] # Supply capacities

n = len(D) # number of demand points
m = len(S) # number of supply points

# cost matrix - cost of shipping from demander i to supplier j
C = [[4, 6, 8, 8],  # Supplier 1 
     [6, 8, 6, 7],  # Supplier 2
     [5, 7, 6, 8]]  # Supplier 3
    #D1 D2 D3 D4

min_cost_alloc, north_west_alloc, vogel_alloc = get_results(C, S, D)

Minimum Cost Method:
Shipping matrix:
[20 20  0  0]
[ 0  0 50 10]
[ 0 10  0 40]
Total cost: 960

North-West Corner Method:
Shipping matrix:
[20 20  0  0]
[ 0 10 50  0]
[ 0  0  0 50]
Total cost: 980

Vogel's Approximation Method:
Shipping matrix:
[20. 20.  0.  0.]
[ 0.  0. 10. 50.]
[ 0. 10. 40.  0.]
Total cost: 920


In [34]:
cost_matrix = np.array(C)
stepping_stone_method(min_cost_alloc, cost_matrix)